In [1]:
from Functions import *
from Optimizations import *
from utils import *
from Saving import *

In [2]:
import time

In [3]:
f = flat_sharp_gaussian
grad_f = grad_flat_sharp_gaussian

In [8]:
def weight_function(U, grad_U, x):
    return np.linalg.norm(grad_U(x), axis=1)

num_particles = 100
process = {}
process["start"] = [[np.random.uniform(-15, 15)] for _ in range(num_particles)]
process["gamma"] = lambda t: 3
process["temperature"] = lambda t: 0.5
process["epsilon"] = 0
process["weight_function"] = weight_function

process["resample_function"] = lambda w, end_p: resample_positions_softmax(w, end_p, beta=-100) #
# process["resample_function"] =  lambda w, end_p: end_p # 
all_paths = diffusion_resampling(f, grad_f, process, 20, 50, domain_enforcer=hyper_cube_enforcer(15, -15, 0.2))


In [9]:
X = np.linspace(-15, 15, 200)
inp = np.array([X])
Y = flat_sharp_gaussian(inp)

all_paths_proc = []

for t in range(len(all_paths)):
    curr_paths = all_paths[t]
    
    all_paths_proc.append([])
    
    for p in range(len(curr_paths)):
        x_curr = np.array(curr_paths[p])
        out = f(x_curr.T)
        all_paths_proc[-1].append(np.concatenate([x_curr, out.reshape(len(out), 1)], axis=1))

all_paths_proc = np.array(all_paths_proc)

In [10]:
ani_path = create_animation_1d_pictures_particles(all_paths_proc, X, Y)

In [11]:
create_animation(ani_path, "test.mp4", framerate=10)